In [40]:
# Notebook to check if the files exist and to get path
# BOTH DC AND CC FILES NEEDS TO UPLOADED TO Files/Monthly Data BEFORE PROCEEDING
# Notebook to transform and combine monthly data
# Combining monthly data to the newtransactiondata table
# DO NOT MANUALLY RUN MULTIPLE TIMES AS IT WOULD APPEND TO THE TABLE - CHANGE PATH
# Automatic schedule is setup with MonthlyDataETL pipeline

from pyspark.sql.functions import *
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql import functions as sf
from pyspark.sql.types import StringType,BooleanType,DateType

df = spark.read.load("Files/csvlist.csv", format="csv", header = False)
df = df.withColumnRenamed("_c0","age")

row_list = df.collect() 

path = "NULL"
ccpath = "NULL"
row = 0
ccrow = 0
flag = 0
count = 0
temppath = "Files/Monthly Data/tempdata"


if mssparkutils.fs.exists(temppath):
    mssparkutils.fs.rm(temppath, True)

n = len(row_list)
for i in range(n):
    if mssparkutils.fs.exists(row_list[i].__getitem__('age')):
        count = count + 1       

if count == 2:
    for i in range(n):
        if mssparkutils.fs.exists(row_list[i].__getitem__('age')):
            if "CC.csv" in row_list[i].__getitem__('age'):
                ccpath = row_list[i].__getitem__('age')
                ccrow = i
            else:
                path = row_list[i].__getitem__('age')
                row = i
    if ccrow == 0 or row == 0:
        print("Error - missing files")
else:
    flag = flag + 1
    path = "NULL"
    ccpath = "NULL"
    print("Error - incorrect number of files")


temppath = "Files/Monthly Data/tempdata"

df = spark.read.load(path, format = "csv", header = True)
df3 = spark.read.load(ccpath, format = "csv", header = True)

df = df.drop("Filter","Description")
df3 = df3.drop('Filter',"Sub-description","Status")
df = df.withColumnsRenamed({"Date":"TransactionDate","Sub-description":"Source",\
                            "Type of Transaction":"TransactionType"})
df3 = df3.withColumnsRenamed({"Date":"TransactionDate","Description":"Source",\
                            "Type of Transaction":"TransactionType"})

df = df.withColumn("AccountType",lit("Debit"))\
       .withColumn("Amount", abs("Amount"))\
       .withColumn("TransactionDate",df.TransactionDate.cast(DateType()))\
       .withColumn("Balance",df.Balance.cast("double"))

df3 = df3.withColumn("AccountType",lit("Credit"))\
         .withColumn("Balance",lit(0)).withColumn("Amount", abs("Amount"))\
         .withColumn("TransactionDate",df3.TransactionDate.cast(DateType()))
         
df3=df3.withColumn("Balance",df3.Balance.cast("double"))

if mssparkutils.fs.exists(temppath):
        mssparkutils.fs.rm(temppath, True)

df.write.format("delta").mode("overwrite").save(temppath)
df3.write.format("delta").mode("append").save(temppath)

df4 = spark.read.parquet(temppath)

food = ['APOS A W', 'COSTCO WHOLESALE','MASALA PAP','HAKKA YETI','UDUPI DOSA'
        'HAKKA YETI','SHELBYS','PEACHYPULP','Swiggy','DOORDASH','Dd'
        ,'A&W','MCDONALD','KFC','TIM HORTONS','EATS','RAJADHANI'
        ,'UBER*','UBEREATS','ZOMATO','KOTHUR','MINT LEAVES','GOURMET'
        ,'MC DONALDS','OPOS DDMINT LEAVES','HOT DOSA','TANDOORI'
        ,'HAKKAKHA','COCHIN DEL','SKIPTHE','BIRYANI','CHARMINAR','CHUCK','ONROUTE'
        ,'SWIGGY','OPOS DD                  VANCO'
        ]
grocery = ["BABU'S", 'BOMBAY SPICES','DARRYLL','SOUTHERN SUPERM'
             ,'BRENT','GROCERYZONE','NOFRILLS','NEPALI BAZAAR'
             ,'FRESHCO','WALMART',"PHIL'S N",'WAL-MART','UNITED SUP','BULK BARN'
]

shopping = ['MACS','PIONEER','BEST BUY','CANADA COMPUTERS','ULTRAMAR'
            ,'CANADIAN TIRE','UPS','DOLLARAMA','STAPLES'
            ,'PLATOS','HASTY','SHOPPERS','HUSKY','DUTY FR'
            ,'SPORTCHEK','NEWEGG','SHELL','INSTACART','AIR-SERV','FACTORY SHOE','COSTCO'
]
weed = ['CANNABIS','VALUE BUDS','FAMOUS FLOWER','CABANA']
parking = ['CANADA WIDE','HONK MOBILE','CITI PLAZA']
haircut = ['AGHA HAIR','HAIR']
movie = ['LANDMARK','IMAGINE CINE','CINEPLEX']
amazon = ['AMAZON','AMZ','Amazon','Amzn']
gas = ['GAS','COSTCO GAS']
alcohol = ['LCBO','BEER','WINE']
microsoft = ['MICROSOFT','MSFT']
iracing = ['IRACING','STEAM']
car = ['BMW','GT TIRES','PART SOURCE','SISAN','BLACKCIRCLES']
activities = ['BOLER','PALASAD','MAKEMYTRIP','CLEARTRIP','REDBUS']
medical = ['COSMO','DENTAL']
airline = ['FLIGHTHUB','GOTOGATE','MARHABA']


regex_food = "|".join(food)
regex_grocery = "|".join(grocery)
regex_shopping = "|".join(shopping)
regex_weed = "|".join(weed)
regex_parking = "|".join(parking)
regex_haircut = "|".join(haircut)
regex_movie = "|".join(movie)
regex_amazon = "|".join(amazon)
regex_alcohol = "|".join(alcohol)
regex_microsoft = "|".join(microsoft)
regex_iracing = "|".join(iracing)
regex_gas = "|".join(gas)
regex_car = "|".join(car)
regex_activities = "|".join(activities)
regex_medical = "|".join(medical)
regex_airline = "|".join(airline)


df4 = df4.withColumn("Source",upper(trim("Source")))
df4 = df4.withColumn("Type", sf.when(df4.Source.rlike(regex_food),"Food")
                            .when(df4.Source.rlike(regex_grocery),"Grocery")
                            .when(df4.Source.rlike(regex_shopping),"Shopping")
                            .when(df4.Source.rlike(regex_weed),"Weed")
                            .when(df4.Source.rlike(regex_parking),"Parking")
                            .when(df4.Source.rlike(regex_haircut),"Haircut")
                            .when(df4.Source.rlike(regex_movie),"Movie")
                            .when(df4.Source.rlike(regex_amazon),"Amazon")
                            .when(df4.Source.rlike(regex_alcohol),"Alcohol")
                            .when(df4.Source.rlike(regex_microsoft),"Microsoft")
                            .when(df4.Source.rlike(regex_iracing),"iRacing")
                            .when(df4.Source.rlike(regex_gas),"Gas")
                            .when(df4.Source.rlike(regex_car),"Car")
                            .when(df4.Source.rlike(regex_activities),"Activities")
                            .when(df4.Source.rlike(regex_medical),"Medical")
                            .when(df4.Source.rlike(regex_airline),"Airline fare")

                            .when(df4.Source.contains("COSTCO WHOLE"),"Costco")
                            .when(df4.Source.contains("WEALTHSIMPLE"),"Investment")
                            .when(df4.Source.contains("LEGACY LEASING"),"Car payment")
                            .when(df4.Source.contains("MONTHLY FEES"),"Monthly Scotiabank fees")
                            .when(df4.Source.contains("ECONOMICAL MUTUAL"),"Car insurance")
                            .when(df4.Source.contains("GOOGLE"),"Google")
                            .when(df4.Source.contains("ENCORE"),"Encore")
                            .when(df4.Source.contains("UBER"),"Uber")
                            .when(df4.Source.contains("ALIEXPRESS"),"Ali Express")
                            .when(df4.Source.contains("REMITLY"),"Remitly")
                            .when(df4.Source.contains("DRAFT PURCHASE"),"Draft purchase")
                            .when(df4.Source.contains("VIRGIN PLUS"),"Mobile bill")
                            .when(df4.Source.contains("ROGERS"),"Mobile bill")
                            .when(df4.Source.contains("UDEMY"),"Udemy")
                            .when(df4.Source.contains("WWW.F1"),"F1")
                            .when(df4.Source.contains("MB-TRANSFER"),"Account transfer")
                            .when(df4.Source.contains("MB-CREDIT"),"Credit card payment")
                            .when(df4.Source.contains("ABM WITHDRAWAL"),"ABM Withdrawal")
                            .when(df4.Source.contains("APPLE STORE"),"Apple")
                            .when(df4.Source.contains("APPLE"),"Apple")
                            .when(df4.Source.contains("CANADA"),"Federal payment")                            
                            .when(df4.Source.contains("INTEREST"),"Interest charge")
                            .when(df4.Source.contains("APPLE BILL"),"iCloud subscription")
                            .when(df4.Source.contains("GARDA"),"Garda pay")
                            .when(df4.Source.contains("TELEPERFORMANCE"),"Teleperformance pay")
                            .when(df4.Source.contains("INTERAC"),"Interac E-Transfer")
                            .when(df4.Source.contains("FEDERAL PAYMENT"),"Federal payment")
                            .when(df4.Source.contains("TEMU"),"Temu")
                            .when(df4.Source.contains("CELPIP"),"Celpip exam")
                            .when(df4.Source.contains("G2A"),"G2A")
                            .when(df4.Source.contains("YOUTUBE PREMIUM"),"Youtube premium subscription")
                            .when(df4.Source.contains("EBAY"),"Ebay")
                            .when(df4.Source.contains("PAYPAL"),"PayPal")
                            .when(df4.Source.contains("RANDSTAD"),"Randstad Pay")
                            .when(df4.Source.contains("FROM - *****09*1980"),"Account Transfer")
                            .when((df4.Source.contains("VALLEY FARM")),"Cigarette")
                            .when(df4.Source.contains("Free Interac"),"Interac E-Transfer")
                            .when(df4.Source.contains("Air India"),"Airline Fare")
                            .when(df4.Source.contains("PAYMENT FROM"),"Credit card payment")
                            .when(df4.Source.contains("Mb-Credit"),"Credit card payment")
                            .when(df4.Source.contains("Remitly"),"Remitly")
                            .when(df4.Source.contains("Canada"),"Federal Payment")
                            .when(df4.Source.contains("Air India"),"Airline Fare")
                            .when(df4.Source.contains("Costco Gas"),"Gas")
                            .when(df4.Source.contains("Amazon"),"Amazon")
                            .when(df4.Source.contains("Amzn"),"Amazon")
                            .when(df4.Source.contains("Monthly Fees"),"Monthly Scotiabank fees")
                            .when(df4.Source.contains("Tim Hortons"),"Food")
                            .otherwise(df4.Type)

                            .otherwise("Not Available") ) \
        .withColumn("LocationId",substring(df4.Source,-5,5))

# Combining monthly data to the newtransactiondata table
# DO NOT RUN MULTIPLE TIMES AS IT WOULD APPEND TO THE TABLE

df4.withColumn("Type",when(df4.Type.isNull(), "Not Available"))
df4 = df4.fillna({"Balance":(0)})
df4.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(temppath)
df4.write.format("delta").mode("append").saveAsTable("newtransactiondata")

if mssparkutils.fs.exists(path):
    mssparkutils.fs.rm(path, True)

if mssparkutils.fs.exists(ccpath):
    mssparkutils.fs.rm(ccpath, True)

if mssparkutils.fs.exists(temppath):
    mssparkutils.fs.rm(temppath, True)

print("Success......")


StatementMeta(, 97a9f673-3eca-4135-a7ec-b5a6814606f0, 51, Finished, Available, Finished)

Success......


In [35]:
from pyspark.sql.functions import *
df = spark.read.table("newtransactiondata")
#var = df.Source.substr((character_length("Source") - 5),5)
df = df.withColumn("Location", substring(df.Source,-5,5))
#display(df.select("Source","Location"))
df = df.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable("newtransactiondata")

StatementMeta(, a9397e55-fad9-4a6d-80c0-a8e68d0e0fda, 37, Finished, Available, Finished)

In [29]:
df = spark.read.table("location")
df = df.withColumn(("CityId"),when(df.city.contains("Windsor"),"WINDS").otherwise(df.CityId))
df.write.format("delta").mode("overwrite").saveAsTable("location")

StatementMeta(, a9397e55-fad9-4a6d-80c0-a8e68d0e0fda, 31, Finished, Available, Finished)

StatementMeta(, 9c6bfff4-f747-4fa8-bbf8-1f127874323b, 11, Finished, Available, Finished)